## 1 第一集 5个实用小技巧

### 1.1 使用map对某些列做特征工程

In [1]:
import pandas as pd

# 生成数据
d = d = {
    "gender": ["male", "female", "male","female"],
    "color": ["red", "green", "blue","green"],
    "age": [25, 30, 15, 32]
}
df = pd.DataFrame(d)
df

,gender,color,age
0,male,red,25
1,female,green,30
2,male,blue,15
3,female,green,32


使用map方法对gender列完成映射, 将原始值映射为其他值，除此之外还可用于行政区划代码的映射，操作会更简便。

In [3]:
code_dict = {'male': 1, 'female': 0}
df['gender_code'] = df['gender'].map(code_dict)
df

,gender,color,age,gender_code
0,male,red,25,1
1,female,green,30,0
2,male,blue,15,1
3,female,green,32,0


### 1.2 使用replace的正则清洗

In [8]:
d = {"customer": ["A", "B", "C", "D"],
"sales":[1100, "950.5RMB", "$400", " $1250.75"]}
df = pd.DataFrame(d)

df["sales"] = df["sales"].replace("[$,RMB]",
                                  "", regex = True).astype("float")
df

,customer,sales
0,A,1100.00
1,B,950.50
2,C,400.00
3,D,1250.75


### 1.3 使用melt对数据透视分析

In [26]:
d = {
    "district_code": [12345, 56789, 101112, 131415],
    "apple": [5.2, 2.4, 4.2, 3.6],
    "banana": [3.5, 1.9, 4.0, 2.3],
    "orange": [8.0, 7.5, 6.4, 3.9]
}
  
df = pd.DataFrame(d)
df

,district_code,apple,banana,orange
0,12345,5.2,3.5,8.0
1,56789,2.4,1.9,7.5
2,101112,4.2,4.0,6.4
3,131415,3.6,2.3,3.9


In [27]:
new_df = df.melt(id_vars='district_code', var_name='fruit_name',
                value_vars=['apple', 'banana', 'orange'], value_name='price')
new_df

,district_code,fruit_name,price
0,12345,apple,5.2
1,56789,apple,2.4
2,101112,apple,4.2
3,131415,apple,3.6
4,12345,banana,3.5
5,56789,banana,1.9
6,101112,banana,4.0
7,131415,banana,2.3
8,12345,orange,8.0
9,56789,orange,7.5


将宽表格合并为长表格，主要的4个参数:
- id_vars: 以此字段为基础进行合并
- var_name: 合并后的字段的名称
- value_vars: 需要合并的字段列表,默认全部合并
- value_names: 合并后的值的名称

### 1.4 已知 year 和 dayofyear，怎么转 datetime

In [15]:
d = {
    "year": [2019, 2019, 2020],
    "day_of_year": [350, 365, 1]
}
df = pd.DataFrame(d)
df

,year,day_of_year
0,2019,350
1,2019,365
2,2020,1


In [20]:
# 创建整数
df['int_number'] = df['year'] * 1000 + df['day_of_year']
print(df)

# 使用to_datetime
df['date'] = pd.to_datetime(df['int_number'], format='%Y%j')
df

   year  day_of_year  int_number                          date
0  2019          350     2019350 1970-01-01 00:00:00.002019350
1  2019          365     2019365 1970-01-01 00:00:00.002019365
2  2020            1     2020001 1970-01-01 00:00:00.002020001


,year,day_of_year,int_number,date
0,2019,350,2019350,2019-12-16
1,2019,365,2019365,2019-12-31
2,2020,1,2020001,2020-01-01


### 1.5 将分类中出现次数较少的值归为 others

In [21]:
d = {"name":['Jone','Alica','Emily','Robert','Tomas',
             'Zhang','Liu','Wang','Jack','Wsx','Guo'],
     "categories": ["A", "C", "A", "D", "A", 
                    "B", "B", "C", "A", "E", "F"]}
df = pd.DataFrame(d)
df

,name,categories
0,Jone,A
1,Alica,C
2,Emily,A
3,Robert,D
4,Tomas,A
5,Zhang,B
6,Liu,B
7,Wang,C
8,Jack,A
9,Wsx,E


In [25]:
# 1. 统计词频, 并归一化
freq = df["categories"].value_counts(normalize=True)
print(freq)

# 2. 过滤出频次较小的值，并得到索引
small_index = freq[freq < 0.1].index
print(small_index)

df["categories"] = df["categories"].replace(small_index, "Others")
df

A    0.363636
C    0.181818
B    0.181818
E    0.090909
D    0.090909
F    0.090909
Name: categories, dtype: float64
Index(['E', 'D', 'F'], dtype='object')


,name,categories
0,Jone,A
1,Alica,C
2,Emily,A
3,Robert,Others
4,Tomas,A
5,Zhang,B
6,Liu,B
7,Wang,C
8,Jack,A
9,Wsx,Others


## 2 第二集 

### 2.1 快速生成时间序列

In [7]:
pd.date_range('2020-01-01', '2020-03-01', freq='m')

DatetimeIndex(['2020-01-31', '2020-02-29'], dtype='datetime64[ns]', freq='M')

### 2.2 重排dataframe的列

In [15]:
import numpy as np
df = pd.DataFrame(np.random.randint(0,20,size=(5,7)),
                  columns=list('ABCDEFG'))
print(df)
df = df[['B', 'C', 'G', 'A', 'D', 'F', 'E']]
print(df)

df.reindex(columns=['B', 'A', 'G', 'C', 'D', 'F', 'E'])
print(df)

    A  B   C  D   E   F  G
0  17  8   2  6   7   4  2
1   1  4  13  3  11   0  6
2   2  8   4  9  10  12  2
3   8  3   8  4   9   9  1
4   6  5  18  1   5  18  3
   B   C  G   A  D   F   E
0  8   2  2  17  6   4   7
1  4  13  6   1  3   0  11
2  8   4  2   2  9  12  10
3  3   8  1   8  4   9   9
4  5  18  3   6  1  18   5
   B   C  G   A  D   F   E
0  8   2  2  17  6   4   7
1  4  13  6   1  3   0  11
2  8   4  2   2  9  12  10
3  3   8  1   8  4   9   9
4  5  18  3   6  1  18   5


### 2.3 调整步长

In [18]:
df = pd.DataFrame(np.random.randint(1,10,size=(240,3)),
    columns = ['商品编码','商品销量','商品库存'])

df.index = pd.date_range('2020-01-01', periods=240, freq='H')
df

,商品编码,商品销量,商品库存
2020-01-01 00:00:00,4,1,4
2020-01-01 01:00:00,5,9,1
2020-01-01 02:00:00,9,6,5
2020-01-01 03:00:00,1,1,6
2020-01-01 04:00:00,2,2,3
...,...,...,...
2020-01-10 19:00:00,6,7,2
2020-01-10 20:00:00,8,7,8
2020-01-10 21:00:00,8,6,5
2020-01-10 22:00:00,5,9,2


In [25]:
day_df = df.resample('D')[['商品库存', '商品销量']].sum()
day_df

,商品库存,商品销量
2020-01-01,108,125
2020-01-02,121,128
2020-01-03,107,132
2020-01-04,137,124
2020-01-05,151,111
2020-01-06,125,100
2020-01-07,116,125
2020-01-08,138,121
2020-01-09,109,95
2020-01-10,114,124


## 3 第四集 
### 3.1 melt的互逆操作

In [32]:
print(new_df)
pivot_df = new_df.pivot(index='district_code', columns='fruit_name', values='price')
pivot_df

    district_code fruit_name  price
0           12345      apple    5.2
1           56789      apple    2.4
2          101112      apple    4.2
3          131415      apple    3.6
4           12345     banana    3.5
5           56789     banana    1.9
6          101112     banana    4.0
7          131415     banana    2.3
8           12345     orange    8.0
9           56789     orange    7.5
10         101112     orange    6.4
11         131415     orange    3.9


fruit_name,apple,banana,orange
district_code,,,
12345,5.2,3.5,8.0
56789,2.4,1.9,7.5
101112,4.2,4.0,6.4
131415,3.6,2.3,3.9
